# Spatial crowdsourcing example

In this example, we will work with *field tasks*. Field tasks are available for performers only in Toloka mobile apps. Requesters set the field tasks, and performers choose them as points on the map. Performers are usually requested to visit the place and check something in person or take photos.

**Examples:**

* monitor prices, products, and outdoor items of interest ([case study](https://toloka.ai/blog/fuel?utm_source=github&utm_medium=site&utm_campaign=tolokakit));
* play the role of *Secret Shopper*: leave reviews for stores and cafés;
* collect data on businesses: whether a particular organization is closed or has changed its office hours;
* monitor promotional events;
* check on outdoor advertising and many more.


>**Note:** For more information about field tasks, check out the Spatial Crowdsourcing page on our [website](https://toloka.ai/usecases/spatialcrowd?utm_source=github&utm_medium=site&utm_campaign=tolokakit) and more precisely [Toloka Requester's Guide](https://toloka.ai/en/docs/guide/tutorials/walk?utm_source=github&utm_medium=site&utm_campaign=tolokakit).

### Call to action
If you found some bugs or have a new feature idea, don't hesitate to [open a new issue on Github](https://github.com/Toloka/toloka-kit/issues/new/choose).
Like our library and examples? Star [our repo on Github](https://github.com/Toloka/toloka-kit)

## Introduction

In this example, we will collect pictures of the Moscow metro entrances.
Because usually spatial crowdsourcing assignments converge longer than others, for this example we have chosen the most frequently used stations. Many potential Toloka performers visit these subways every day.


This example also can be reused for production tasks such as monitoring the state of objects, checking the presence of an organization or other physical object.

For example to check:

- that the area around the store is clean and tidy,
- that the store has placed a new ad at the entrance,
- how your advertisement was placed on the billboards,
- the cleanliness of the waste disposal areas,
- that the bench is installed correctly in the right place.

## Data preparation

In most cases, field tasks are tied to a specific point on the map. In our case, these are subway entrances. The performer must come to the entrance and complete some actions.

There is a way to set these points on the map through [toloka-kit](https://toloka.ai/en/docs/toloka-kit/?utm_source=github&utm_medium=site&utm_campaign=tolokakit).

But first, let's take an example of the Okhotny Ryad metro station, from which it is easy to get to the Red Square and Kremlin. Let's open the maps in [Yandex](https://yandex.com/maps/213/moscow/search/metro%20Okhotny%20Ryad/?ll=37.617426%2C55.756762&sctx=ZAAAAAgBEAAaKAoSCfFmDd5XtSVAEXmxMEROl0hAEhIJAAAAAIAdV0ARe2X%2FuFTeQ0AoCjgAQNytB0gBVc3MzD5qAnJ1cACdAc3MTD2gAQCoAQC9AVmBJ6HCARH6tInp5AKeqLT7A%2Bi%2FyPT6BuoBAPIBAPgBAIICEm1ldHJvIE9raG90bnkgUnlhZIoCAA%3D%3D&sll=37.617426%2C55.756762&sspn=0.007170%2C0.002595&z=17.41) or [Google](https://www.google.com/maps/place/Okhotnyy+ryad/@55.7574454,37.615033,18z/data=!4m5!3m4!1s0x46b54a5adb3b8d5b:0x22be0270b70ed47d!8m2!3d55.757689!4d37.6164801). And find the required entrances.

You will see something like this:

<table  align="center">
  <tr><td>
    <img src="./img/google_map1.png"
         alt="Okhotny Ryad on Google Maps"  width="1000">
  </td></tr>
  <tr><td align="center">
    <b>Figure 1.</b> Okhotny Ryad on Google Maps
  </td></tr>
  <tr><td><br><br></td></tr>
  <tr><td>
    <img src="./img/yandex_map1.png"
         alt="Okhotny Ryad on Yandex Maps"  width="1000">
  </td></tr>
  <tr><td align="center">
    <b>Figure 2.</b> Okhotny Ryad on Yandex Maps
  </td></tr>
</table>

Now, to get the point on the map:

1. Find the 1st entrance to the Okhotny Ryad metro station - a red-letter 'M' with a number 1 on the left side of the screen.

2. *On Google Maps:* right-click on the entrance point and coordinates will immediately appear in a pop-up menu.

    *On Yandex Maps:* click next to the entrance point, then click on the pop-up card 'Tverskaya Street,1' and coordinates will appear in the left menu.

>**Note:** It is also helpful to switch maps to satellite mode or check yourself by panorama views.

<table  align="center">
  <tr><td>
    <img src="./img/google_map2.png"
         alt="Coordinates on Google Maps"  width="1000">
  </td></tr>
  <tr><td align="center">
    <b>Figure 3.</b> Coordinates on Google Maps
  </td></tr>
  <tr><td><br><br></td></tr>
  <tr><td>
    <img src="./img/yandex_map2.png"
         alt="Coordinates on Yandex Maps"  width="1000">
  </td></tr>
  <tr><td align="center">
    <b>Figure 4.</b> Coordinates on Yandex Maps
  </td></tr>
</table>

We will get the following coordinates: 55.756916, 37.614546.

These are latitude and longitude geographic coordinates expressed as [decimal fractions of a degree](https://en.wikipedia.org/wiki/Decimal_degrees).

## Create a project

### Set up the environment
Specifically, we will use the following libraries:

* `toloka-kit` to develop main Toloka functionalities;
* `pandas` and `numpy` to perform data manipulation;
* `ipyplot` to deal with images in Jupyter Notebooks.

In [ ]:
%%capture
!pip install toloka-kit==0.1.26
!pip install pandas
!pip install ipyplot

import datetime
import io
import logging
import sys
import getpass

import ipyplot
import pandas
from PIL import Image

import toloka.client as toloka
import toloka.client.project.template_builder as tb

In [ ]:
logging.basicConfig(
    format='[%(levelname)s] %(name)s: %(message)s',
    level=logging.INFO,
    stream=sys.stdout,
)

Create a `TolokaClient` PRODUCTION or SANDBOX instance. All API calls will pass through it.

Obtain your [OAuth token](https://toloka.ai/en/docs/api/concepts/access?utm_source=github&utm_medium=site&utm_campaign=tolokakit#access__token) from Toloka or [Toloka Sandbox](https://toloka.ai/en/docs/guide/concepts/sandbox?utm_source=github&utm_medium=site&utm_campaign=tolokakit).

In [ ]:
toloka_client = toloka.TolokaClient(getpass.getpass('Enter your OAuth token: '), 'PRODUCTION') # Or switch to 'SANDBOX'
print(toloka_client.get_requester())

### Create a dataset
We will use a ready-made dataset of Moscow metro entrances and select several stations from the circular (koltsevaya) line.

This dataset is compiled by the Toloka team and is distributed under the Creative Commons Attribution 4.0 international license
[![License: CC BY 4.0](https://img.shields.io/badge/License-CC%20BY%204.0-lightgrey.svg)](https://creativecommons.org/licenses/by/4.0/)


In [ ]:
!curl https://tlk.s3.yandex.net/dataset/moscow_metro_entrance_2020_en.tsv --output dataset.tsv
dataset = pandas.read_csv('dataset.tsv', sep='\t')
dataset = dataset[dataset['line'].isin(['Koltsevaya line'])].sample(frac=1).iloc[:5]
print(dataset)

>**Note:** Dataset contains not only the coordinates of the stations but also their names. In field tasks, the performer will come to the real object only once and must clearly understand what is required to do there.
> It is always helpful to give as much information as possible so that the performer does not make a mistake.

For example, attach a reference photo: from which angle you want to capture the entrance to the subway station.

Otherwise, the performer may misunderstand the task and capture the object too close or too shallow. For example, capture only the door when we need a general plan, or capture a general building plan when we want a sign with the opening hours of the organization.

### Input and output data formats

We need to decide what kind of data we have as input and what data we want to receive at the output.

At the input, we have:

- coordinates of the object (entrance);
- station name and entrance number.

At the output, we want to get:

- verdict - if the object was found or not;
- photos of the place itself, if the object is found;
- photos of the surrounding area, if the object is not found - to make sure that the performer was where;
- the coordinates of the performer at the time of task execution to check that it was the right place.

In [ ]:
input_specification = {
    'coordinates': toloka.project.StringSpec(),
    'entrance': toloka.project.StringSpec(),
}
output_specification = {
    'verdict': toloka.project.StringSpec(),
    'entrance_images': toloka.project.ArrayFileSpec(required=False),
    'around_images': toloka.project.ArrayFileSpec(required=False),
    'performer_coordinates':  toloka.project.CoordinatesSpec(current_location=True, required=False),
    'comment': toloka.project.StringSpec(required=False),
}

### Create instructions for performers

It is essential to prepare detailed instructions covering all the corner cases.

Let's upload a prepared instruction from an HTML file and then analyze best practices for writing instruction for a task.

In [ ]:
prepared_instruction = open('instruction.html').read().strip()

For field tasks, a good start for the instruction is configuring a device.


<table  align="center">
  <tr><td>
    <img src="./img/instruction1.png"
         alt="How to configure device"  width="800">
  </td></tr>
  <tr><td align="center">
    <b>Figure 5.</b> How to configure device
  </td></tr>
</table>

Next, we describe the steps for completing the task.

Be sure to provide exact steps what performers should do if something went wrong: there is no requested object or limited access to it, or something else went wrong.

<table  align="center">
  <tr><td>
    <img src="./img/instruction2.png"
         alt="Steps for performers"  width="800">
  </td></tr>
  <tr><td align="center">
    <b>Figure 6.</b> Steps for performers
  </td></tr>
</table>

Then, it is always helpful to include requirements for photographs or other information you require from the performer.

Usually, these requirements match with rules on how you will check the task.

<table  align="center">
  <tr><td>
    <img src="./img/instruction3.png"
         alt="Photo requirements"  width="800">
  </td></tr>
  <tr><td align="center">
    <b>Figure 7.</b> Photo requirements
  </td></tr>
</table>

Another best practice is to provide examples of correctly taken photos and photos that will not be accepted in the task.

<table  align="center">
  <tr><td>
    <img src="./img/instruction4.png"
         alt="Photo examples"  width="800">
  </td></tr>
  <tr><td align="center">
    <b>Figure 8.</b> Photo examples
  </td></tr>
</table>

### Project interface

In the cell below, we define some UI elements of the project, such as the task's title, links, text fields, buttons, conditions. We use [template builder](https://toloka.ai/en/docs/toloka-kit/source/toloka.client.project.template_builder?utm_source=github&utm_medium=site&utm_campaign=tolokakit) (`tb`) instance from `toloka-kit`.

>**Note:** You can also create an interface for this task in [Template Builder](https://clck.ru/SMCHv) web version using our [documentation](https://toloka.ai/en/docs/template-builder/?utm_source=github&utm_medium=site&utm_campaign=tolokakit).

In [ ]:
# Project title
header = tb.MarkdownViewV1('# Cleanliness at the entrance to the metro:\n---')
# Name of metro station and number of the entrance
entrance_name = tb.TextViewV1(tb.InputData('entrance'), label='Entrance name:')
# Situation on the spot
# If the required entrance was found or not
workflow_options = tb.ButtonRadioGroupFieldV1(
    tb.OutputData('verdict'),
    [
        tb.GroupFieldOption('ok', 'I found the right entrance'),
        tb.GroupFieldOption('no_obj', 'The required entrance is not there'),
    ],
    validation=tb.RequiredConditionV1(hint='Choose one of the answer options'),
)

# Task interface if the required object was found by a performer
# First, we set all the elements that will be shown

# Description on what to do next
exist_header = tb.MarkdownViewV1('** Photos of the entrance from the right and left sides **\n\n_Take two photos of the entrance on the right and on the left. The photo should show the entire entrances and the floor. So that you can assess the cleanliness around the entrance to the metro._')

# Links to examples
example_links = tb.LinkGroupViewV1(
    [
        tb.LinkGroupViewV1.Link(
            'https://tlk.s3.yandex.net/toloka-kit/images_for_instructions/0spatial_good1.png',
            'Example1',
        ),
        tb.LinkGroupViewV1.Link(
            'https://tlk.s3.yandex.net/toloka-kit/images_for_instructions/0spatial_good2.png',
            'Example2',
        ),
        tb.LinkGroupViewV1.Link(
            'https://tlk.s3.yandex.net/toloka-kit/images_for_instructions/0spatial_good3.png',
            'Example3',
        ),
    ]
)

# Field for loading responses
image_loader = tb.MediaFileFieldV1(
    tb.OutputData('entrance_images'),
    tb.MediaFileFieldV1.Accept(photo=True, gallery=True),
    multiple=True,
    validation=tb.RequiredConditionV1(hint='There must be at least 2 photos of the entrance: from the right and from the left'),
)

# Define the condition by which all the necessary interface elements will be shown
exist_ui = tb.IfHelperV1(
    tb.EqualsConditionV1('ok', tb.OutputData('verdict')),
    tb.ListViewV1([exist_header, example_links, image_loader]),
)

# Task interface if the required object was NOT found by a performer
# First, we set all the elements that will be shown

# Description on what to do next
miss_header = tb.MarkdownViewV1('**Take 4 photos in all directions**\n\n_So that we can understand where you are and that there is no entrance to the metro here._')

# Field for loading responses
miss_image_loader = tb.MediaFileFieldV1(
    tb.OutputData('around_images'),
    tb.MediaFileFieldV1.Accept(photo=True, gallery=True),
    multiple=True,
    validation=tb.RequiredConditionV1(hint='There must be at least 4 photos of the place'),
)

# Define the condition by which all the necessary interface elements will be shown
miss_ui = tb.IfHelperV1(
    tb.EqualsConditionV1('no_obj', tb.OutputData('verdict')),
    tb.ListViewV1([miss_header, miss_image_loader]),
)

# Validation of the performer's geolocation
coordinates_validation = tb.AllConditionV1(
    [
        # Check that geolocation reading is generally possible
        tb.RequiredConditionV1(
            tb.OutputData('performer_coordinates'),
            hint="Couldn't get your coordinates. Please enable geolocation.",
        ),
        # Check that the performer is close enough to the required location
        tb.DistanceConditionV1(
            tb.LocationData(),
            tb.InputData('coordinates'),
            max=500,
            hint='You are too far from the entrance.',
        ),
    ]
)

# Plugin to make tasks look nice in the performer's interface
task_width_plugin = tb.TolokaPluginV1(
    layout = tb.TolokaPluginV1.TolokaPluginLayout('scroll', task_width=400)
)

# Plugin for writing device coordinates at the time of task's execution to the output
coordinates_save_plugin = tb.TriggerPluginV1(
    fire_immediately=True,
    action=tb.SetActionV1(
        tb.OutputData('performer_coordinates'),
        tb.LocationData()
    ),
)

# How performers will see the task
project_interface = toloka.project.TemplateBuilderViewSpec(
    view=tb.ListViewV1(
        [header, entrance_name, workflow_options, exist_ui, miss_ui],
        validation=coordinates_validation,
    ),
    plugins=[task_width_plugin, coordinates_save_plugin]
)

print('interface prepared')

The performer will see the interface like this:

<table  align="center">
  <tr>
  <td>
    <img src="./img/performer_interface1.jpg"
         alt="Photo examples"  width="350">
  </td>
  <td>
    <img src="./img/performer_interface2.jpg"
         alt="Photo examples"  width="350">
  </td>
  <td>
    <img src="./img/performer_interface3.jpg"
         alt="Photo examples"  width="350">
  </td>
  </tr>
  <td colspan="3" align="center">
    <b>Figure 9.</b> How performer will see your task
  </td></tr>
</table>

### Create a project in Toloka

Finally, we can create an instance of the [Project class](https://toloka.ai/en/docs/toloka-kit/source/toloka.client.project#module-toloka.client.project?utm_source=github&utm_medium=site&utm_campaign=tolokakit) and send it to Toloka or Toloka Sandbox.

The project with all the instructions and interface will appear in your [Toloka Requester's account](https://platform.toloka.ai/requester?utm_source=github&utm_medium=site&utm_campaign=tolokakit).

In [ ]:
project = toloka.Project(
    public_name='Cleanliness of metro entrances',
    public_description='Take two photos of the entrance on the right and on the left. The photo should show the entire entrances and the floor. So that you can assess the cleanliness around the entrance to the metro.',
    public_instructions=prepared_instruction,
    # We indicate that this task is selected by the performer on the map.
    # Tasks without reference to the map are displayed simply as a list.
    assignments_issuing_type='MAP_SELECTOR',
    # We will indicate how the title of the task and description will be displayed on the map
    assignments_issuing_view_config=toloka.Project.AssignmentsIssuingViewConfig(
        title_template='Photo metro entrance',  # Set title as a constant
        description_template='{{inputParams["entrance"]}}',  # Set description from the input parameters
                                                            # That way we can have different description for each point on the map
    ),
    task_spec=toloka.project.task_spec.TaskSpec(
        input_spec=input_specification,
        output_spec=output_specification,
        view_spec=project_interface
    ),
)

project = toloka_client.create_project(project)

## Create a pool

Here we create an instance of the [Pool class](https://toloka.ai/en/docs/toloka-kit/source/toloka.client.pool?utm_source=github&utm_medium=site&utm_campaign=tolokakit) and send it to Toloka or Toloka Sandbox.

The pool is a set of tasks sent out to performers. Learn more about working with a [pool](https://toloka.ai/en/docs/guide/concepts/pool-main?utm_source=github&utm_medium=site&utm_campaign=tolokakit).

In [ ]:
pool = toloka.Pool(
    project_id=project.id,
    private_name='Metro entrances',
    may_contain_adult_content=False,
    will_expire=datetime.datetime.utcnow() + datetime.timedelta(days=10),
    reward_per_assignment=2,
    assignment_max_duration_seconds=60 * 60 * 2,  # We give 2 hours to complete the task,
                                                  # So that performer has time to book a task, get there and complete it.
    auto_accept_solutions=False,    # We will check the completed tasks manually before paying for them.
    auto_accept_period_day=5,       # Number of days to determine if we pay
    # Only performers from the Toloka mobile application are allowed to pick field tasks
    filter=(
        (toloka.filter.ClientType == 'TOLOKA_APP') &
        (toloka.filter.Languages.in_('EN')) &
        (toloka.filter.RegionByPhone.in_(225)) &  # Russia
        (toloka.filter.RegionByIp.in_(213))  # Moscow
    ),
    defaults=toloka.Pool.Defaults(
        # Overlap is not used for most field tasks
        default_overlap_for_new_task_suites=1,
        default_overlap_for_new_tasks=1,
    ),
)

pool = toloka_client.create_pool(pool)

## Add tasks and run the pool

To add tasks and bind them to map coordinates, you need to create instances of [TaskSuite](https://toloka.ai/en/docs/toloka-kit/source/toloka.client?utm_source=github&utm_medium=site&utm_campaign=tolokakit#module-toloka.client.task_suite) class. Only `TaskSuites` allow bindings with geolocation.

In [ ]:
task_suites = [
    toloka.TaskSuite(
        pool_id=pool.id,
        latitude=round(row['latitude'], 10),  # First number in a pair of coordinates
        longitude=round(row['longitude'], 10),  # Second number
        overlap=1,
        tasks=[
            toloka.Task(input_values={
                'entrance': row['name'],
                'coordinates': f"{round(row['latitude'], 10)}, {round(row['longitude'], 10)}"
            })
        ],
    )
    for i, row in dataset.iterrows()]

task_suites = toloka_client.create_task_suites(task_suites)

pool_id = pool.id
pool = toloka_client.open_pool(pool.id)

## Get results and check them

Validation of field tasks differs from validation of other types of tasks.

We suggest using the following rules for field task validation:

1. First, you need to check that the performer came where you wanted. We did this in the task interface by comparing the device coordinates with the required coordinates.

    But it will be helpful to cross-check that the user has attached real photographs of the place you required. For example, check panorama views on Yandex or Google Maps. Or an old photo of the same place, if you have one.

2. Secondly, you need to remember what you pay the performer, even if the object does not exist on the spot. It is not the fault of the performer.

    The performer came and verified this information for you, and for that work should be paid with the same reward.

3. In the third, field tasks usually converge much slower, and there is no point in waiting for the entire pool to close. We suggest retrieving completed tasks from the pool and sending them for verification periodically.

    This allows you to collect information faster. And the performers get the well-deserved reward more quickly.

If you interrupted the execution of this notebook, uncomment the code below and include your `Pool ID`.





In [ ]:
# pool_id = '23461858'

Let's get all completed but not yet verified tasks from our pool:

In [ ]:
results_list = []

for assignment in toloka_client.get_assignments(pool_id=pool_id, status='SUBMITTED'):
    for task, solution in zip(assignment.tasks, assignment.solutions):
        results_list.append({
            'assignment_id': assignment.id,
            'input_values': task.input_values,
            'output_values': solution.output_values
        })

print(f'New results received: {len(results_list)}' if len(results_list) > 0 else 'Not received any new results yet, try to run this cell later.')

results_iter = iter(results_list)

If you get at least one result, you can run the cells below. In the first one, you see all the information received. In the second and third, you can accept or reject the assignment.

You can execute these cells several times.

In [ ]:
res = next(results_iter, None)

if res is not None:
    images = []
    for id in res['output_values']['entrance_images']:
        out_b = io.BytesIO()
        toloka_client.download_attachment(id, out_b)
        images.append(Image.open(out_b).convert('RGBA'))

    print(f"Entrance name:\t\t{res['input_values']['station']}")
    print(f"Object found:\t\t{res['output_values']['verdict']}")
    print(f"Object coordinates:\t{res['input_values']['coordinates']}")
    print(f"Performer coordinates:\t{res['output_values']['performer_coordinates']}")
    ipyplot.plot_images(
        images,
        max_images=10,
        img_width=600,
    )
else:
    print('No more results')

Now let's try to reject or accept the completed assignment.

Remember that it will be automatically accepted after the time specified when creating the pool.

If you try to accept or reject the same task twice, you will get an error.

In [ ]:
# If you want to accept assignment
if res is not None:
    updated_assignment = toloka_client.accept_assignment(res['assignment_id'], 'Well done!')
    print(updated_assignment.status)

In [ ]:
# If you want to reject assignment
if res is not None:
    updated_assignment = toloka_client.reject_assignment(res['assignment_id'], 'Type your issue here')
    print(updated_assignment.status)

>**Note:** You do not have to check all the photos you received by yourself. You can also validate field tasks through Toloka. Check how it's done in our [object detection example.](https://github.com/Toloka/toloka-kit/tree/main/examples/1.computer_vision/object_detection) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Toloka/toloka-kit/blob/main/examples/1.computer_vision/object_detection/object_detection.ipynb)

## Summary
We have considered the simple example of a field task. In fact, with field tasks, you can carry out various projects: from a Secret Shopper to monitoring the state of urban infrastructure.

The main takeaway is not to forget about the specifics of such projects: give the most detailed instructions to the performers and carefully check the results.